# 01 - Data Cleaning Notebook - Isailton 

- Project: Machine Learning
- Dataset: Student performance (stundents-mat.csv)
- Team MIKE WHEELER (Safina, Charles, Isailton)

## Step 0 — Import Libraries & Load Data

In [1]:
# Step 0 - Import required libraries
# pandas: data manipulation and analysis
# numpy: numerical computations

import pandas as pd
import numpy as np

In [2]:
# Step 0 - Load the dataset using a RELATIVE PATH
# This ensures the code works for all team members after git push
# The dataset uses ';' as a separator

DATA_PATH = "../data/student-mat.csv"

df = pd.read_csv(DATA_PATH, sep=";")

# Display first rows to confirm successful loading
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


## Step 1 — Check Shape of the Data

In [3]:
# Step 1 - Check the shape of the dataset
# This shows how many rows (observations) and columns (features) we have

df.shape

# Explanation:
# - Rows → number of students
# - Columns → number of features

(395, 33)

In [4]:
# Step 2 - Display original column names
df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')

In [5]:
# Step 2 - Rename columns following PEP8 conventions
# - Convert to lowercase
# - Replace spaces with underscores

df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

# Confirm column names were updated
df.columns

# Why this matters:
# - Standardized column names improve readability and prevent coding errors.

Index(['school', 'sex', 'age', 'address', 'famsize', 'pstatus', 'medu', 'fedu',
       'mjob', 'fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'dalc',
       'walc', 'health', 'absences', 'g1', 'g2', 'g3'],
      dtype='object')

## Step 3 — Check Data Types

In [6]:
# Step 3 - Inspect data types of each column
# This helps decide how to clean and preprocess each feature

df.dtypes

# Explanation:
# - object → categorical variables
# - int64 / float64 → numerical variables

school        object
sex           object
age            int64
address       object
famsize       object
pstatus       object
medu           int64
fedu           int64
mjob          object
fjob          object
reason        object
guardian      object
traveltime     int64
studytime      int64
failures       int64
schoolsup     object
famsup        object
paid          object
activities    object
nursery       object
higher        object
internet      object
romantic      object
famrel         int64
freetime       int64
goout          int64
dalc           int64
walc           int64
health         int64
absences       int64
g1             int64
g2             int64
g3             int64
dtype: object

## Step 4 — Check for Missing (NaN) Values

In [7]:
# Step 4 - Count missing values per column

df.isna().sum()

school        0
sex           0
age           0
address       0
famsize       0
pstatus       0
medu          0
fedu          0
mjob          0
fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
dalc          0
walc          0
health        0
absences      0
g1            0
g2            0
g3            0
dtype: int64

In [8]:
# Step 4 - Sort missing values for easier inspection

df.isna().sum().sort_values(ascending=False)

# Explanation:
# - This dataset has very few or no missing values, making it ideal for ML.

school        0
sex           0
age           0
address       0
famsize       0
pstatus       0
medu          0
fedu          0
mjob          0
fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
dalc          0
walc          0
health        0
absences      0
g1            0
g2            0
g3            0
dtype: int64

## Step 5 — Check and Remove Duplicates

In [9]:
# Step 5 - Check how many duplicated rows exist

df.duplicated().sum()

np.int64(0)

In [10]:
# Step 5 - Remove duplicated rows (if any)

df = df.drop_duplicates()

# Why this is important:
# - Duplicates can bias model training and evaluation.

## Step 6 — Split Dataset into Categorical and Numerical Features

In [11]:
# Step 6 - Identify categorical columns (non-numeric)

categorical_cols = df.select_dtypes(include="object").columns

# Step 6 - Identify numerical columns

numerical_cols = df.select_dtypes(include=["int64", "float64"]).columns

categorical_cols, numerical_cols

# Explanation:
# - Different data types require different preprocessing strategies.

(Index(['school', 'sex', 'address', 'famsize', 'pstatus', 'mjob', 'fjob',
        'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
        'nursery', 'higher', 'internet', 'romantic'],
       dtype='object'),
 Index(['age', 'medu', 'fedu', 'traveltime', 'studytime', 'failures', 'famrel',
        'freetime', 'goout', 'dalc', 'walc', 'health', 'absences', 'g1', 'g2',
        'g3'],
       dtype='object'))

## Step 7 — Clean Categorical Features

### Step 7.1 — Explore Unique Values (EDA)

In [12]:
# Step 7.1 - Explore unique values in each categorical column
# This helps identify inconsistencies or typos

for col in categorical_cols:
    print(f"Column: {col}")
    print(df[col].unique())
    print("-" * 40)

# Explanation:
# - This step is a basic EDA technique to understand categorical distributions.



Column: school
['GP' 'MS']
----------------------------------------
Column: sex
['F' 'M']
----------------------------------------
Column: address
['U' 'R']
----------------------------------------
Column: famsize
['GT3' 'LE3']
----------------------------------------
Column: pstatus
['A' 'T']
----------------------------------------
Column: mjob
['at_home' 'health' 'other' 'services' 'teacher']
----------------------------------------
Column: fjob
['teacher' 'other' 'services' 'health' 'at_home']
----------------------------------------
Column: reason
['course' 'other' 'home' 'reputation']
----------------------------------------
Column: guardian
['mother' 'father' 'other']
----------------------------------------
Column: schoolsup
['yes' 'no']
----------------------------------------
Column: famsup
['no' 'yes']
----------------------------------------
Column: paid
['no' 'yes']
----------------------------------------
Column: activities
['no' 'yes']
-----------------------------------

### Step 7.2 — Handle Missing Values in Categorical Columns

In [13]:
# Step 7.2 - Fill missing categorical values with the mode (most frequent value)

for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Why mode?
# - It preserves the most common category without distorting data.

## Step 8 — Clean Numerical Features

### Step 8.1 — Descriptive Statistics (EDA)

In [14]:
# Step 8.1 - Generate summary statistics for numerical features

df[numerical_cols].describe()

# What we learn here:
# - Min / Max values
# - Mean & median
# - Potential outliers

,age,medu,fedu,traveltime,studytime,failures,famrel,freetime,goout,dalc,walc,health,absences,g1,g2,g3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


### Step 8.2 — Handle Missing Numerical Values

In [15]:
# Step 8.2 - Fill missing numerical values with the median
# Median is robust against outliers

for col in numerical_cols:
    df[col] = df[col].fillna(df[col].median())

## Step 9 — Final Data Validation

In [16]:
# Step 9 - Final overview of the cleaned dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   pstatus     395 non-null    object
 6   medu        395 non-null    int64 
 7   fedu        395 non-null    int64 
 8   mjob        395 non-null    object
 9   fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

In [17]:
# Step 9 - Ensure no missing values remain

df.isna().sum().sum()

# Expected result: 0

np.int64(0)

## Step 10 — Save Cleaned Dataset

In [18]:
# Step 10 - Save the cleaned dataset for next steps (feature engineering & modeling)

df.to_csv("../data/01a_students_mat_cleaned_isailton.csv", index=False)

# Why this step matters:
# - Keeps cleaning separate from modeling and ensures reproducibility.